In [ ]:
!pip install plotly

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext

import pandas as pd

In [ ]:
spark = SparkSession\
            .builder\
            .master("spark://spark-master:7077")\
            .config("spark.sql.repl.eagerEval.enabled", "true")\
            .config('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener')\
            .config('spark.driver.extraClassPath', '/opt/bitnami/python/lib/python3.6/site-packages/sparkmonitor/listener_2.12.jar')\
            .getOrCreate()

In [ ]:
pdf = pd.read_json("arxiv-metadata-oai-snapshot.json", lines=True, nrows=100)
df = spark.createDataFrame(pdf)

In [ ]:
df

In [ ]:
df_drop_report_no = df.drop("report-no")

In [ ]:
# negated
df_nonull_comments = df.filter(~(F.col("comments") == "null"))

In [ ]:
df_w_pagenumber = df_nonull_comments.select(F.split(F.col("comments"), " ")[0].alias("pagenumber"))

In [ ]:
df_submitter = df.groupBy(F.col("submitter")).count().orderBy(F.col("count").desc()).limit(5)

In [ ]:
df_submitter

In [ ]:
pdf_submitter = df_submitter.toPandas()

In [ ]:
import plotly.express as px

In [ ]:
px.bar(pdf_submitter, x="submitter", y="count")

In [ ]:
df_datetime = df.withColumn("update_date", F.to_timestamp(F.col("update_date"), 'yyyy-MM-dd'))

In [ ]:
df_math_pysics = df.filter((F.lower(F.col("categories")).contains("math") | F.lower(F.col("categories")).contains("physic")))

In [ ]:
df_math_pysics

In [ ]:
spark.stop()